In [3]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from docx import Document
from docx.shared import Inches

base_folder = r"Plant_Images\Day11_Morning\Health"

image_paths = [
    os.path.join(base_folder, "Plant1_Day11_Morning_Side.jpg"),
    os.path.join(base_folder, "Plant2_Day11_Morning_Side.jpg"),
    os.path.join(base_folder, "Plant3_Day11_Morning_Side.jpg")
]

word_doc_path = os.path.join(base_folder, "Plant_Color_Analysis_Side.docx")

def keep_only_green(image_path):
    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Could not load {image_path}")
        return None, None, None, None, None
    
    # Convert to RGB and HSV
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Define green color range in HSV
    lower_green = np.array([35, 40, 40])  # Adjust for best results
    upper_green = np.array([90, 255, 255])

    # Create mask for green areas
    green_mask = cv2.inRange(image_hsv, lower_green, upper_green)

    # Apply erosion to remove small noise
    kernel = np.ones((5,5), np.uint8)
    green_mask = cv2.erode(green_mask, kernel, iterations=2)

    # Apply mask to keep only green parts
    green_only_image = cv2.bitwise_and(image_rgb, image_rgb, mask=green_mask)

    # Resize images to reduce size
    new_size = (250, 250)  
    resized_original = cv2.resize(image_rgb, new_size)
    resized_green_only = cv2.resize(green_only_image, new_size)

    combined_image = np.hstack((resized_original, resized_green_only))

    # Compute average RGB values for green regions
    mean_r = np.mean(image_rgb[:, :, 0][green_mask > 0])
    mean_g = np.mean(image_rgb[:, :, 1][green_mask > 0])
    mean_b = np.mean(image_rgb[:, :, 2][green_mask > 0])
    
    return combined_image, mean_r, mean_g, mean_b, os.path.basename(image_path)

doc = Document()
doc.add_heading("Plant Color Analysis Report", level=1)

# Process images and collect color data
results = []

for image_path in image_paths:
    result_image, r, g, b, image_name = keep_only_green(image_path)
    if result_image is not None:
        results.append((image_name, r, g, b))

        # Convert the processed image to a format that can be inserted into the Word document
        temp_image_path = os.path.join(base_folder, f"temp_{image_name}")
        cv2.imwrite(temp_image_path, cv2.cvtColor(result_image, cv2.COLOR_RGB2BGR))

        # Add image to Word document
        doc.add_paragraph(f"Original (Left) vs Processed (Right): {image_name}")
        doc.add_picture(temp_image_path, width=Inches(4.5)) 

        # Remove the temporary image file after inserting into the document
        os.remove(temp_image_path)

# Plot color distribution as a bar graph
plt.figure(figsize=(8, 5))
bar_width = 0.2
x = np.arange(len(results))

plt.bar(x - bar_width, [r for _, r, _, _ in results], bar_width, color='r', label="Red")
plt.bar(x, [g for _, _, g, _ in results], bar_width, color='g', label="Green")
plt.bar(x + bar_width, [b for _, _, _, b in results], bar_width, color='b', label="Blue")

plt.xticks(x, [img for img, _, _, _ in results], rotation=15)
plt.title("Color Distribution in Green Areas")
plt.xlabel("Images")
plt.ylabel("Average Color Intensity")
plt.legend()
plt.tight_layout()

# Save the graph as a temporary image for the Word document
temp_graph_path = os.path.join(base_folder, "temp_color_graph.jpg")
plt.savefig(temp_graph_path)
plt.close()

# Add graph to Word document
doc.add_paragraph("Color Distribution Graph:")
doc.add_picture(temp_graph_path, width=Inches(5))

os.remove(temp_graph_path)

# Save the Word document inside the "Health" folder
doc.save(word_doc_path)

print(f"Word document saved at: {word_doc_path}")


Word document saved at: Plant_Images\Day11_Morning\Health\Plant_Color_Analysis_Side.docx
